In [36]:
import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import spacy
import re

In [37]:
# from google.colab import drive
# drive.mount('/content/drive')

In [38]:
# !ls -r

In [39]:
# cd drive/MyDrive/

In [40]:
# !ls -r

In [41]:
# cd CodaBench_Sem_Eval

In [42]:
#  cd val

In [48]:
# GPU/CPU Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Data
train = pd.read_csv('../public_data_test/track_a/train/eng.csv')
val = pd.read_csv('../public_data_test/track_a/dev/eng.csv')
test = pd.read_csv('../public_data_test/track_a/test/eng.csv')
emotions = ["anger", "fear", "joy", "sadness", "surprise"]

# Initialize BERT Tokenizer & Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
nlp = spacy.load("en_core_web_sm")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [44]:
val

,id,text,anger,fear,joy,sadness,surprise
0,eng_dev_track_a_00001,Older sister (23 at the time) is a Scumbag Stacy.,1,0,0,0,0
1,eng_dev_track_a_00002,"And I laughed like this: garhahagar, because m...",0,1,0,0,0
2,eng_dev_track_a_00003,It overflowed and brown shitty diarrhea water ...,1,1,0,1,1
3,eng_dev_track_a_00004,Its very dark and foggy.,0,1,0,0,0
4,eng_dev_track_a_00005,"Then she tried to, like, have sex with/strangl...",1,1,0,0,1
...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,My heart was beating fast from excitement.,0,0,1,0,0
112,eng_dev_track_a_00113,A fraying rope stretches down from the rafters.,0,1,0,0,1
113,eng_dev_track_a_00114,so i cried my eyes out and did the drawing.,0,0,0,1,0
114,eng_dev_track_a_00115,Never been so close to a group ass-wooping in ...,1,1,0,0,1


In [45]:
# Preprocessing Function
def pre_process(text):
    text = re.sub(r"[.,;:!?'\"“”()]", "", text)  # Remove punctuation
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
    return encoded_input['input_ids'].squeeze(0).to(device)

# Convert Text to BERT Embeddings
def get_bert_embeddings(texts):
    embeddings = []
    for text in texts:
        input_ids = pre_process(text).unsqueeze(0)
        with torch.no_grad():
            outputs = bert_model(input_ids)
        embeddings.append(outputs.last_hidden_state[:, 0, :].cpu().numpy())  # Extract [CLS] token
    return np.vstack(embeddings)

X_train = get_bert_embeddings(train["text"])
X_val = get_bert_embeddings(val["text"])

# POS Feature Extraction
def get_pos_features(texts):
    return [[token.pos_ for token in nlp(text)] for text in texts]

train_pos_tags = get_pos_features(train["text"])
val_pos_tags = get_pos_features(val["text"])

# Convert POS Tags to Indices
pos_vocab = {pos: idx for idx, pos in enumerate(set(tag for tags in train_pos_tags for tag in tags))}
train_pos_indices = [[pos_vocab[tag] for tag in tags] for tags in train_pos_tags]
val_pos_indices = [[pos_vocab.get(tag, 0) for tag in tags] for tags in val_pos_tags]

# Pad POS Sequences to Fixed Length
max_length = max(max(len(seq) for seq in train_pos_indices), max(len(seq) for seq in val_pos_indices))
train_pos_indices = [seq + [0] * (max_length - len(seq)) for seq in train_pos_indices]
val_pos_indices = [seq + [0] * (max_length - len(seq)) for seq in val_pos_indices]

# Convert to PyTorch Tensors
train_pos_indices = torch.tensor(train_pos_indices, dtype=torch.long).to(device)
val_pos_indices = torch.tensor(val_pos_indices, dtype=torch.long).to(device)

In [46]:
# Trainable POS Embedding Layer
class POSEmbedding(nn.Module):
    def __init__(self, num_pos_tags, embedding_dim):
        super(POSEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=num_pos_tags, embedding_dim=embedding_dim)

    def forward(self, pos_indices):
        return self.embedding(pos_indices)

pos_embedding_layer = POSEmbedding(len(pos_vocab), embedding_dim=16).to(device)

# Model Definition
class EmotionClassifier(nn.Module):
    def __init__(self, bert_dim=768, pos_dim=16, hidden_dim=128, output_dim=5):
        super(EmotionClassifier, self).__init__()
        self.fc1 = nn.Linear(bert_dim + pos_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, bert_embeddings, pos_indices):
        pos_embeds = pos_embedding_layer(pos_indices).mean(dim=1)  # Average POS embeddings
        combined_features = torch.cat((bert_embeddings, pos_embeds), dim=1)
        x = self.relu(self.fc1(combined_features))
        return self.fc2(x)

# Initialize Model
model = EmotionClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [49]:
# Prepare Training Data
y_train = torch.tensor(train[emotions].values, dtype=torch.float32).to(device)
y_val = torch.tensor(val[emotions].values, dtype=torch.float32).to(device)

train_features = torch.tensor(X_train, dtype=torch.float32).to(device)
val_features = torch.tensor(X_val, dtype=torch.float32).to(device)

dataset = TensorDataset(train_features, train_pos_indices, y_train)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Training Loop
epochs = 400
losses = []

for epoch in tqdm(range(epochs + 1), desc="Training Loop"):
    model.train()
    for features, pos_indices, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(features, pos_indices)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss: {round(loss.item(), 3)}")
        torch.save(model.state_dict(), f'./08-02-25/net_epoch_{epoch}.pth')
        losses.append(round(loss.item(), 3))

print(f"Final Loss after {epochs} epochs: {losses[-1]}")

# Prediction Function
def get_predictions(X_val, pos_indices, model, threshold=0.5):
    model.eval()
    with torch.no_grad():
        yhat = torch.sigmoid(model(X_val, pos_indices)).cpu().numpy()
    return yhat > threshold

# Generate Predictions for Multiple Epochs
for i in range(5):
    epoch = i * 100
    model.load_state_dict(torch.load(f'./08-02-25/net_epoch_{epoch}.pth'))
    y_pred = get_predictions(val_features, val_pos_indices, model, 0.45)

    val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])
    val_data_with_pred = val_data_with_pred.astype(int)
    val_data_with_pred['id'] = val['id']
    val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

    formatted_time = datetime.datetime.now().strftime('%Y-%m-%d_%H_%M_%S')
    val_data_with_pred.to_csv(f'../results/alt_exp_4/alt_exp_4_epoch_{epoch}_pred_eng_a_{formatted_time}.csv', index=False)

    print(val_data_with_pred)

Training Loop:   0%|          | 1/401 [00:00<02:02,  3.27it/s]

Epoch 0: Loss: 0.544


Training Loop:  25%|██▌       | 101/401 [00:32<01:31,  3.29it/s]

Epoch 100: Loss: 0.43


Training Loop:  50%|█████     | 201/401 [01:04<01:00,  3.31it/s]

...

Training Loop:  75%|███████▌  | 301/401 [01:35<00:32,  3.03it/s]

Epoch 300: Loss: 0.489


Training Loop: 100%|██████████| 401/401 [02:07<00:00,  3.15it/s]

Epoch 400: Loss: 0.356
Final Loss after 400 epochs: 0.356
                        id  Anger  Fear  Joy  Sadness  Surprise
0    eng_dev_track_a_00001      0     1    0        0         0
1    eng_dev_track_a_00002      0     1    0        0         0
2    eng_dev_track_a_00003      0     1    0        0         0
3    eng_dev_track_a_00004      0     1    0        0         0
4    eng_dev_track_a_00005      0     1    0        0         0
..                     ...    ...   ...  ...      ...       ...
111  eng_dev_track_a_00112      0     1    0        0         0
112  eng_dev_track_a_00113      0     1    0        0         0
113  eng_dev_track_a_00114      0     1    0        0         0
114  eng_dev_track_a_00115      0     1    0        0         0
115  eng_dev_track_a_00116      0     1    0        0         0

[116 rows x 6 columns]
                        id  Anger  Fear  Joy  Sadness  Surprise
0    eng_dev_track_a_00001      0     1    0        0         1
1    eng_dev_track_a_0


<ipython-input-49-f75ae65d54db>:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./08-02-25/net_epoch_{epoch}.pth'))
<ipython-input-49-f

In [50]:
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import numpy as np

def evaluate(y_true, y_pred):
    jaccard = jaccard_score(y_true, y_pred, average='samples')
    print(f"\nJaccard Score: {round(jaccard, 4)}")

    for avg in ['micro', 'macro']:
        recall = recall_score(y_true, y_pred, average=avg, zero_division=0)
        precision = precision_score(y_true, y_pred, average=avg, zero_division=0)
        f1 = f1_score(y_true, y_pred, average=avg, zero_division=0)
        print(f"{avg.upper()} - Recall: {round(recall, 4)}, Precision: {round(precision, 4)}, F1: {round(f1, 4)}")

def evaluate_per_class(y_true, y_pred, class_names):
    print("\nDetailed Per-Class Evaluation:\n" + "="*40)

    recall = recall_score(y_true, y_pred, average=None, zero_division=0)
    precision = precision_score(y_true, y_pred, average=None, zero_division=0)
    f1 = f1_score(y_true, y_pred, average=None, zero_division=0)

    for i, class_name in enumerate(class_names):
        print(f"\n*** {class_name.capitalize()} ***")
        print(f"  Recall: {recall[i]:.4f}")
        print(f"  Precision: {precision[i]:.4f}")
        print(f"  F1 Score: {f1[i]:.4f}")

# Define y_test (ground truth labels from the validation set)
y_test = val[emotions].values  # Convert DataFrame to NumPy array
y_test = np.nan_to_num(y_test, nan=0)

# Define y_pred using the trained model
y_pred = get_predictions(val_features, val_pos_indices, model, threshold=0.45).astype(int)  # Ensure binary output

print(y_test.shape, y_pred.shape)

# # Run evaluation
evaluate(y_test, y_pred)
evaluate_per_class(y_test, y_pred, class_names=emotions)

# Evaluate each epoch
# for epoch, y_pred in predictions_by_epoch.items():
#     print(f"--- Epoch {epoch} ---")
#     evaluate(y_true, y_pred)
#     evaluate_per_class(y_true, y_pred)


(116, 5) (116, 5)

Jaccard Score: 0.4784
MICRO - Recall: 0.5795, Precision: 0.6755, F1: 0.6239
MACRO - Recall: 0.5106, Precision: 0.6563, F1: 0.5596

Detailed Per-Class Evaluation:

*** Anger ***
  Recall: 0.2500
  Precision: 0.6667
  F1 Score: 0.3636

*** Fear ***
  Recall: 0.7778
  Precision: 0.7424
  F1 Score: 0.7597

*** Joy ***
  Recall: 0.5806
  Precision: 0.6000
  F1 Score: 0.5902

*** Sadness ***
  Recall: 0.4286
  Precision: 0.5769
  F1 Score: 0.4918

*** Surprise ***
  Recall: 0.5161
  Precision: 0.6957
  F1 Score: 0.5926


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
val[emotions]

,anger,fear,joy,sadness,surprise
0,1,0,0,0,0
1,0,1,0,0,0
2,1,1,0,1,1
3,0,1,0,0,0
4,1,1,0,0,1
...,...,...,...,...,...
111,0,0,1,0,0
112,0,1,0,0,1
113,0,0,0,1,0
114,1,1,0,0,1


In [53]:
from sklearn.metrics import classification_report

In [54]:
print("\nFinal Validation Performance with Best Thresholds:")
print(classification_report(
    y_test,
    y_pred,
    target_names=emotions
))


Final Validation Performance with Best Thresholds:
              precision    recall  f1-score   support

       anger       0.67      0.25      0.36        16
        fear       0.74      0.78      0.76        63
         joy       0.60      0.58      0.59        31
     sadness       0.58      0.43      0.49        35
    surprise       0.70      0.52      0.59        31

   micro avg       0.68      0.58      0.62       176
   macro avg       0.66      0.51      0.56       176
weighted avg       0.67      0.58      0.61       176
 samples avg       0.60      0.56      0.55       176



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
